In [2]:
import geopandas as gpd
from shapely import Point
import pandas as pd

gdf_boundaries = gpd.read_file("./metadata/living_pop_boundary.geojson")
gdf_boundaries.loc[:,"TOT_REG_CD"] = gdf_boundaries.loc[:,"TOT_REG_CD"].astype(str)
gdf_boundaries.crs = "EPSG:5179"
gdf_boundaries = gdf_boundaries.to_crs("EPSG:4326")

In [3]:
df_population = pd.read_csv("./raw_data/LOCAL_PEOPLE_20240101.csv", encoding="cp949")
df_population = df_population.loc[:,['집계구코드', '총생활인구수']].groupby('집계구코드').mean().reset_index()
df_population = df_population.rename(columns={"총생활인구수": "defacto_pop"})
df_population.loc[:,'집계구코드'] = df_population.loc[:,'집계구코드'].astype(str)

In [4]:
gdf_population = gdf_boundaries.merge(df_population, left_on="TOT_REG_CD", right_on="집계구코드", how="left")

In [5]:
df_points = pd.read_json("./sample/sample_coord.json", encoding="utf-8")
points = df_points[['latitude', 'longitude']].apply(
lambda row: Point(row['longitude'], row['latitude']), axis=1)
gdf_points = gpd.GeoDataFrame(df_points, geometry=points, crs="EPSG:4326")

for index, row in gdf_points.iterrows():
    point = row['geometry']
    try:
        gdf_points.at[index, 'defacto_pop'] = gdf_population[gdf_population.geometry.contains(point)]['defacto_pop'].values[0]
    except IndexError:
        ## the nearest polygon
        nearest_polygon = gdf_population.distance(point).nsmallest(1).index[0]
        gdf_points.at[index, 'defacto_pop'] = gdf_population.loc[nearest_polygon, 'defacto_pop']


C:\Users\seung\AppData\Local\Temp\ipykernel_31572\585127956.py:12: UserWarning: Geometry is in a geographic CRS. Results from 'distance' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  nearest_polygon = gdf_population.distance(point).nsmallest(1).index[0]
C:\Users\seung\AppData\Local\Temp\ipykernel_31572\585127956.py:12: UserWarning: Geometry is in a geographic CRS. Results from 'distance' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  nearest_polygon = gdf_population.distance(point).nsmallest(1).index[0]
C:\Users\seung\AppData\Local\Temp\ipykernel_31572\585127956.py:12: UserWarning: Geometry is in a geographic CRS. Results from 'distance' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  nearest_polygon = gdf_population.distance(point).nsmallest(1).index[0]
C:\Users\seung\AppData\Lo

In [6]:
gdf_points.loc[:,['latitude', 'longitude', 'defacto_pop', 'temp_celsius', 'humidity']].to_json("./sample/sample_coord.json", orient="records")

In [7]:
gdf_points.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 4008 entries, 0 to 4007
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype   
---  ------        --------------  -----   
 0   latitude      4008 non-null   float64 
 1   longitude     4008 non-null   float64 
 2   defacto_pop   4008 non-null   float64 
 3   temp_celsius  4008 non-null   float64 
 4   humidity      4008 non-null   float64 
 5   geometry      4008 non-null   geometry
dtypes: float64(5), geometry(1)
memory usage: 348.2 KB
